In [700]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import random as rn
import matplotlib.pyplot as plt

In [593]:
data = pd.read_csv('Train_Kaggle_Data_11_12PM.csv')
data['MSSubClass'] = data['MSSubClass'].astype(str)
data['OverallQual'] = data['OverallQual'].astype(str)
data['OverallCond'] = data['OverallCond'].astype(str)

In [541]:
features.reset_index(inplace=True)

In [543]:
#MasVnrType
import scipy.stats as stats

In [768]:
pairs = []
cats = []
sum_ = 0
for cat in cat_var:
    
    category = data[cat]
    uniq_lvls = list(set(category))
    Price_lvls = list(range(len(uniq_lvls)))
    
    for level in range(len(uniq_lvls)): #create a list with all the Sales Prices per level = Price_lvls
        Price_lvls[level] = list(data[category == uniq_lvls[level]]['SalePrice'])
    
    if stats.f_oneway(*Price_lvls).pvalue < 0.05: #making sure the category is significant 
    
        Bonferroni = len(uniq_lvls) #coeff to divide alpha by
    
        for i in range(len(uniq_lvls)-1): #retain all pairs that are significant when performing a t test
            for l in range(i+1,len(uniq_lvls)):
                if stats.ttest_ind(Price_lvls[i],Price_lvls[l]).pvalue < 0.05/Bonferroni:
                    pairs.append(cat + '_' + uniq_lvls[i])
                    cats.append(cat + '_' + uniq_lvls[i])
                    cats.append(cat)
                    
factors_to_keep = []
unique_pairs = set(pairs)
for i in unique_pairs:
    for cat in cat_var:
        if cat in i:
            nb = cats.count(cat)
            nb = 0.4 * nb
            if pairs.count(i)>nb:
                factors_to_keep.append(i)
len(factors_to_keep) #112 factors are significant  
    



#obtain a data set with all the significant factors to be used to identify highest frquency dummies to be dropped     
all_col = list(data.columns) + factors_to_keep
col_keep = list(set(all_col) - set(cat_var))
data3 = pd.get_dummies(data)
filter_col = [col for col in data3 if col in col_keep]
data3 = data3[filter_col]


#find, per category, if there are more than two significant levels in that category, which level has the highest count
#to eliminate it for proper dummification

M =[]
for category in cat_var:
    L= []
    
    sum_ = 0
    for subset in range(len(filter_col)):
        sum_ += filter_col[subset].count(category)
    
    if sum_ > 1:
        for subset in range(len(filter_col)):
            if category in filter_col[subset]:
                L = L + list([filter_col[subset]])
                
        N = list(range(len(L)))
    
        for feature in range(len(L)):
            N[feature] = sum(data3[L[feature]])
        m = max(N)
        M += list([L[N.index(m)]])

#we wanted to substract the list of factors in M, from the set of all significant factors

C:\Users\trist\Desktop\CondaFile\lib\site-packages\numpy\core\fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\trist\Desktop\CondaFile\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\trist\Desktop\CondaFile\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\trist\Desktop\CondaFile\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\trist\Desktop\CondaFile\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [769]:
cont_var = list(['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','BsmtFullBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch','PoolArea'])

categ_var = list(set(factors_to_keep) - set(M)) #unique significant levels - levels with highest frequency

factors_kept = cont_var + categ_var + list(['SalePrice'])

subset_factors = pd.get_dummies(data)

filter1 = [col for col in subset_factors if col in factors_kept]

data_factors_kept = subset_factors[filter1]

data_factors_kept.to_csv('Factor_Selection_F_Reg_Anova.csv')